Initiliaze and prepare all models

In [1]:
# ~~~~~~~~~~~~~~~~
# PyTorch Model

import torch
import yaml
from models.decoder import architectures

with open("models/decoder/sequential_decoder_params.yaml") as file:
    params = yaml.safe_load(file)

decoder = architectures.LightningNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
decoder.load_state_dict(torch.load("models/decoder/sequential_decoder_length20.pth"))
decoder.eval();

# ~~~~~~~~~~~~~~~~
# ESM

# To Do

### Data & Metrics

Specify data

In [2]:
from torch.utils import data
from data.gdm_dataset import GasDataSet

dataset = GasDataSet("data/30x25/test.pt", seq_len=20)

Specify metrics

In [3]:
import gdm_metrics

rmse = {"decoder": 0,
        }

kld = {"decoder": 0,
       }

### Run

In [4]:
from tqdm import tqdm

list_of_models = [
                  "decoder",
                 ]

dataloader = iter(data.DataLoader(dataset, shuffle=False, drop_last=True))

for X, y in tqdm(dataloader):
    with torch.no_grad(): 
        if "decoder" in list_of_models:
            y_decoder = decoder(X.squeeze(2))
            rmse["decoder"] += gdm_metrics.rmse(y_decoder, y)
            kld["decoder"] += gdm_metrics.kld(y_decoder, y)

100%|████████████████████████████████████| 30120/30120 [03:38<00:00, 138.01it/s]


### Evaluation

In [5]:
# Print results
print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataloader)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataloader)}")

########
# RMSE #
########
decoder: 	 0.07327050715684891
########
# KL D #
########
decoder: 	 0.003545727115124464
